## The Basis of Symbolic-Numeric Integration

In this section, we informally introduce the symbolic-numeric integration algorithm, as implemented by **SymbolicNumericIntegraion.jl**.

Let's start with a simple example, $f(x) = x \sin x$, and show how to integrate it using the *method of indeterminate coefficients*. The main idea is to write the solution, i.e., $S = \int x \sin x\,dx$, as a sum of multiple possible terms with unknown coefficients,

\begin{equation}
  S = \sum_i q_i \mathbb{T}_i(x)
  \,,
  \tag{1}
\end{equation}

where $q_i$ are constant coefficients and $\mathbb{T}_i(x)$ are *reasonable candidate* terms. For our first example, and considering that $(\sin x)' = \cos x$ and $(\cos x)' = -\sin x$, a reasonable set of terms is $\mathbb{T} = \{x, \sin x, \cos x, x\sin x, x\cos x\}$. Of course, we need a better method to find $\mathbb{T}$ than saying it should be a reasonable set! In fact, we will discuss this problem is details later, but for now assume that an oracle provides $\mathbb{T}$. We have

\begin{equation}
  S = q_1 x + q_2 \sin x  + q_3 \cos x + q_4 x \sin x + q_5 x \cos x
  \,.
  \tag{2}
\end{equation}

Differentiating with respect to $x$,

\begin{equation}
  S' = q_1 + (q_4 - q_3) \sin x + (q_2 + q_5) \cos x - q_5 x \sin x + q_4 x \cos x
  \,.
  \tag{3}
\end{equation}

By definition, $\int S\,dx = f$; therefore, $S' = f = x \sin x$ (note that, as it is customary in symbolic integration, we ignore the constant inegration term). We obtain the following linear system,

\begin{equation}
  \begin{array}{ll}
    q_1 = 0 \\
    q_4 - q_3 = 0 \\
    q_2 + q_5 = 0 \\
    -q_5 = 1 \\
    q_4 = 0  
  \end{array}  
  \tag{4}
\end{equation}


Solving the linear the system, we find $q_5 = -1$, $q_2 = 1$, and $q_1 = q_3 = q_4 = 0$. Therefore,

\begin{equation}
  S = \int x \sin x\,dx = \sin x - x \cos x 
  \,.
  \tag{5}
\end{equation}

As it should be.

Note that the preceding calculations were all essentially symbolic and there was no need for numerical computation. However, this is not always the case. Let's look at another example. This time, let $f(x) = \sin^2 x$. We assume that the oracle, who knows the correct answer $\int \sin^2 x = (x - \sin x\cos x)/2$, gives us $\mathbb{T} = \{x, \sin x\cos x\}$ (in practice, the list will be longer, but we use the abbreviated one to reduce clutter). Following the same process as before,

\begin{equation}
  S = q_1 x + q_2 \sin x\cos x
  \,,
  \tag{6}
\end{equation}

and,

\begin{equation}
  S' = q_1 + q_2 \cos^2 x - q_2\sin^2 x
  \,.
  \tag{7}
\end{equation}

Equating $S'$ to $\sin^2 x$, we get

\begin{equation}
  \begin{array}{ll}
    q_1 = 0 \\
    q_2 = 0 \\
    q_2 = -1
  \end{array}  
  \tag{8}
\end{equation}

which is a contradiction. We can resolve this problem by using $\sin^2 x + \cos^2 x = 1$ to write

\begin{equation}
  S' = q_1 + q_2 (1 - \sin^2 x) - q_2\sin^2 x =
       (q_1 + q_2) - 2q_2 \sin^2 x
  \,.
  \tag{9}
\end{equation}

Therefore,

\begin{equation}
  \begin{array}{ll}
    q_1 + q_2 = 0 \\
    -2q_2 = 1
  \end{array}  
  \tag{10}
\end{equation}

Finally, we have the correct answer $q_1 = 1/2$ and $q_2 = -1/2$.

Numerical computation becomes necessary partly due to the limitations of **JuliaSymbolics** in converting expressions into unique *canonical* forms. Therefore, identities like $\sin^2 x + \cos^2 x = 1$ (and may more, some subtle and some complex) may not be correctly applied. In fact, the problem is more fundamental and according to the Richardson's theorem, the problem of finding canonical forms of transcendental expressions is undecided. 

Another reason for using numerical computation is that the list of candidates may not be (and usually is not) linearly-independent. Finding a linearly-independent subset of a set of expressions using symbolical computation is a very difficult problem but can be done numerically. 

The next example show cases the problems of linear dependence. Let $f(x) = \sinh x\cosh x$. Assume that the oracle returns the following candidate list (which is typical of such lists),

\begin{equation}
    \mathbb{T} = \{\cosh^2 x, \cosh x\sinh x, \sinh2 x, x\cosh^2 x, x\cosh x\sinh x, x\sinh^2 x\}
    \,.
    \tag{14}
\end{equation}

If we follow the same procedure described above, a singular matrix error occurs. The reason is the fact that $\cosh^2 x - \sinh^2 x = 1$; therefore, $x\cosh^2 x$ and $x\sinh^2 x$ are linearly dependent. The solution is to prune $\mathbb{T}$ to a linearly-independent subset,

\begin{equation}
    \mathbb{T} = \{\cosh^2 x, \cosh x\sinh x, x\cosh^2 x, x\cosh x\sinh x\}
    \,,
    \tag{15}
\end{equation}

Now, we can calculate the correct answer $\int \sinh x\cosh\,dx = \frac{1}{2}\cosh^2 x$.